In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [9]:
import os
import pandas as pd
import glob
import pickle

os.sys.path.insert(0, '../evaluation')

from evaluate import evaluate_all_systems, preprocess_all_models

In [16]:
models = [os.path.basename(p) for p in glob.glob(f'../data/models/dev/*')]

dfs = []

for model in models:
    if os.path.isfile(f'../data/models/dev/{model}/system_evaluation.csv'):
        df_ = pd.read_csv(f'../data/models/dev/{model}/system_evaluation.csv', index_col=['subset', 'references', 'metric'])
        dfs.append(df_)

scores_df = pd.concat(dfs, keys=models).reset_index(level=[1, 2], drop=True).unstack()
scores_df.columns = scores_df.columns.droplevel()

In [17]:
params_dfs = []
for model in models:
    with open(f'../data/models/dev/{model}/params.pkl', 'rb') as f:
        params_dfs.append(pd.DataFrame([pickle.load(f)], index=[model]))
    

params_df = pd.concat(params_dfs)

In [18]:
df = pd.merge(scores_df, params_df, left_index=True, right_index=True)

cols = ['bleu', 'meteor', 'ter', 'dp_scorer', 'max_dp', 'sa_scorer', 'max_sa', 'tems_lm_name', 'tems_lm_n', 'max_tems', 'referrer', 'txs_lm_name', 'txs_lm_n']
df.sort_values('bleu', ascending=False).loc[:, cols]

,bleu,meteor,ter,dp_scorer,max_dp,sa_scorer,max_sa,tems_lm_name,tems_lm_n,max_tems,referrer,txs_lm_name,txs_lm_n
3029073323364641550,59.52,0.442747,0.362394,ltr_lasso,2,ltr_lasso,10,lower,6,3,counter,lower,6
1695230394238246047,59.06,0.440958,0.368147,ltr_lasso,2,random,3,lower,3,3,counter,lower,6
-5765958883275118294,58.91,0.441617,0.372730,ltr_lasso,2,ltr_lasso,3,lower,6,3,counter,lower,6
-2628958327849352414,58.74,0.440756,0.368586,ltr_lasso,2,random,3,lower,6,3,counter,lower,6
-4100986303314228521,58.73,0.440268,0.369463,ltr_lasso,2,ltr_lasso,3,lower,3,3,counter,lower,6
4589235621195969825,58.46,0.438335,0.369853,ltr_lasso,2,ltr_lasso,3,lower,6,3,counter,lower,3
-79971058925217812,58.37,0.436115,0.371365,ltr_lasso,2,ltr_lasso,3,lower,3,3,counter,lower,3
-5847135404309646138,58.30,0.439581,0.375411,ltr_lasso,2,inv_ltr_lasso,3,lower,6,3,counter,lower,6
-8633909619867043180,58.29,0.436343,0.367659,ltr_lasso,2,random,3,lower,3,3,counter,lower,3
-3625841863022073959,58.14,0.437389,0.368391,ltr_lasso,2,random,3,lower,6,3,counter,lower,3
